Tic Tac Toe Reinforcement Learning by Mike Donahoe

In [0]:
new_board = [[0, 0, 0], 
             [0, 0, 0], 
             [0, 0, 0]]

In [0]:
## converts board to hashable int to be used in dictionary
## board[2][2] is the first digit and board[0][0] is the last digit
def convertBoardToHashable(board):
  ret = 0
  multiplier = 1
  for row in range(3):
    for cell in range(3):
      if not (row == 0 and cell == 0):
        multiplier *= 10
      ret += board[row][cell] * multiplier
  return ret

In [0]:
import copy 

## flips 1s and 2s 
def flipBoard(board):
  new_board = copy.deepcopy(board)
  for row in range(3):
    for cell in range(3):
      if board[row][cell] == 0:
        new_board[row][cell] = 0
      elif board[row][cell] == 1:
        new_board[row][cell] = 2
      else:
        new_board[row][cell] = 1
  return new_board

In [0]:
##convert space list to int to be used in dictionary
def convertSpaceToHashable(space):
  ret = 0
  if space == [0,0]:
    ret = 1
  if space == [0,1]:
    ret = 2
  if space == [0,2]:
    ret = 3
  if space == [1,0]:
    ret = 4
  if space == [1,1]:
    ret = 5
  if space == [1,2]:
    ret = 6
  if space == [2,0]:
    ret = 7
  if space == [2,1]:
    ret = 8
  if space == [2,2]:
    ret = 9
  return ret

In [0]:
## dictionary
##   1. hashed board is key
##   2. sub_dictionary is value
##      a. hashed coordinate is key
##      b. reward is value

reward_table = {}

In [0]:
def checkWinner(board):

  ## check rows ##
  if board[0][0] == board[0][1] == board[0][2] != 0: 
    return board[0][0]
  elif board[1][0] == board[1][1] == board[1][2] != 0: 
    return board[1][0]
  elif board[2][0] == board[2][1] == board[2][2] != 0: 
    return board[2][0]

  ## check columns ##
  elif board[0][0] == board[1][0] == board[2][0] != 0: 
    return board[0][0]
  elif board[0][1] == board[1][1] == board[2][1] != 0: 
    return board[0][1]
  elif board[0][2] == board[1][2] == board[2][2] != 0: 
    return board[0][2]

  ## check diagonals ##
  elif board[0][0] == board[1][1] == board[2][2] != 0:
    return board[0][0]
  elif board[2][0] == board[1][1] == board[0][2] != 0:
    return board[2][0]
  
  ## if no winner, check for tie
  else:
    tie = True
    for row in range(3):
      for cell in range(3):
        if board[row][cell] == 0:
          tie = False
          break
      if tie == False:
        break
    if tie == True:
      return 0
    else: 
      return -1  #game not over

In [0]:
def getEmptySpaces(board):
  spaces = []
  for row in range(3):
    for square in range(3):
      if board[row][square] == 0:
        spaces.append( convertSpaceToHashable([row, square]) )
  return spaces

In [0]:
import random

def botAct(board, reward_table):
    
    hashed_board = convertBoardToHashable(board)    

    ## if greed <= 3, pick a random action
    greed = random.randint(0,10)

    ## bot has already seen the game state and picked every move at least once so it will pick the best move
    if hashed_board in reward_table and len(reward_table[hashed_board]) == 9 and greed > 3:
      avail_moves = reward_table[hashed_board]

      max_reward = -99999999999999999999999
      best_move = 0
      for move in avail_moves:
        if avail_moves[move] > max_reward:
          max_reward = avail_moves[move]
          best_move = move
      return best_move

    ## bot is faced with new game state and will pick a random move, or greed < 3 and bot will explore
    else:
      empty_spaces = getEmptySpaces(board)
      random_index = random.randint(0, len(empty_spaces)-1 )
      random_move = empty_spaces[random_index]
      return random_move

In [0]:
def playGame(board, reward_table):


  p1_history = [] ## [hashed board, move], hashed board always uses 1 to indicate acting player and 2 to indicate opponent
  p2_history = [] ##

  player = 1

  while True:

    ##get move and update moves list
    if player == 1:
      selected_move = botAct(board, reward_table)
      hashed_board = convertBoardToHashable(board)

      p1_history.append([hashed_board, selected_move])
    
    if player == 2:
      flipped_board = flipBoard(board)
      selected_move = botAct(flipped_board, reward_table)
      board

      hashed_board = convertBoardToHashable(flipped_board)
      p2_history.append([hashed_board, selected_move])

    ## update board
    if selected_move == 1:
      board[0][0] = player
    if selected_move == 2:
      board[0][1] = player
    if selected_move == 3:
      board[0][2] = player
    if selected_move == 4:
      board[1][0] = player
    if selected_move == 5:
      board[1][1] = player
    if selected_move == 6:
      board[1][2] = player
    if selected_move == 7:
      board[2][0] = player
    if selected_move == 8:
      board[2][1] = player
    if selected_move == 9:
      board[2][2] = player

    ## check for winner and assign rewards/penalties
    if checkWinner(board) == -1:
      if player == 1:
        player = 2
      else:
        player = 1

      board
      continue
      
    if checkWinner(board) == 0:

      ## assign tie rewards
      for entry in p1_history:
        state = entry[0]
        move = entry[1]     

        try:
          rewards_dict = reward_table[state]
          try:
            old_reward = rewards_dict[move]
          except KeyError: ## bot has not made move within state
            old_reward = 0
          rewards_dict.update({move : old_reward + .1})
          reward_table.update({state: rewards_dict})

        except KeyError: ## bot has not seen state yet
          reward_table.update({state : {move : .1}})
        

      for entry in p2_history:
        state = entry[0]
        move = entry[1]  

        try:
          rewards_dict = reward_table[state]
          try:
            old_reward = rewards_dict[move]
          except KeyError: ## bot has not made move within state
            old_reward = 0
          rewards_dict.update({move : old_reward + .1})
          reward_table.update({state: rewards_dict})

        except KeyError: ## bot has not seen state yet
          reward_table.update({state : {move : .1}})

      break

    if checkWinner(board) == 1:

      ## assign rewards to p1
      for entry in p1_history:
        state = entry[0]
        move = entry[1]     

        try:
          rewards_dict = reward_table[state]
          try:
            old_reward = rewards_dict[move]
          except KeyError: ## bot has not made move within state
            old_reward = 0
          rewards_dict.update({move : old_reward + 1})
          reward_table.update({state: rewards_dict})

        except KeyError: ## bot has not seen state yet
          reward_table.update({state : {move : 1}})  

      ## assign penalties to p2
      for entry in p2_history:
        state = entry[0]
        move = entry[1] 

        try:
          rewards_dict = reward_table[state]
          try:
            old_reward = rewards_dict[move]
          except KeyError: ## bot has not made move within state
            old_reward = 0
          rewards_dict.update({move : old_reward - 1})
          reward_table.update({state: rewards_dict})

        except KeyError: ## bot has not seen state yet
          reward_table.update({state : {move : -1}})

      break

    if checkWinner(board) == 2:

      ## assign penalties to p1
      for entry in p1_history:
        state = entry[0]
        move = entry[1]     

        try:
          rewards_dict = reward_table[state]
          try:
            old_reward = rewards_dict[move]
          except KeyError: ## bot has not made move within state
            old_reward = 0
          rewards_dict.update({move : old_reward - 1})
          reward_table.update({state: rewards_dict})

        except KeyError: ## bot has not seen state yet
          reward_table.update({state : {move : -1}})  

      ## assign rewards to p2
      for entry in p2_history:
        state = entry[0]
        move = entry[1]  

        try:
          rewards_dict = reward_table[state]
          try:
            old_reward = rewards_dict[move]
          except KeyError: ## bot has not made move within state
            old_reward = 0
          rewards_dict.update({move : old_reward + 1})
          reward_table.update({state: rewards_dict})

        except KeyError: ## bot has not seen state yet
          reward_table.update({state : {move : 1}})        
      break

In [0]:
count = 0

while count < 10000000:
  board = copy.deepcopy(new_board)
  playGame(board, reward_table)
  count+=1

In [24]:
reward_table

{0: {1: 186730.70000023153,
  2: 113592.50000012649,
  3: 189239.10000023432,
  4: 1882310.3000366476,
  5: 271683.0000001915,
  6: 114375.50000012692,
  7: 188126.80000023285,
  8: 112890.90000012623,
  9: 186670.80000023296},
 1000002: {2: 2691.599999999909,
  3: 4004.49999999997,
  4: -519.4999999999797,
  5: 5068.100000000013,
  6: 1243.7999999999954,
  8: 2493,
  9: 4653.1999999999825},
 101002002: {2: 915.1000000000031,
  3: 1076,
  5: 896.4000000000032,
  6: 926.0000000000035,
  8: 1256},
 121002102: {2: 950, 5: 986, 6: 956},
 2000000: {1: -18121.299999999363,
  2: -27704.100000001636,
  3: -17237.79999999779,
  4: -28857.300000005416,
  5: -6800.399999999003,
  6: -28107.90000000177,
  8: -29292.600000005845,
  9: -18421.799999999414},
 202000001: {2: -1287,
  3: -517.3999999999913,
  4: -2498.4000000000146,
  5: -1683.3000000000186,
  6: -1967.7000000000235,
  8: -695.099999999988},
 202001201: {2: -958, 5: -595, 6: -640, 8: -961},
 100002000: {1: 4540.599999999989,
  2: 1779.